In [1]:
!git clone https://github.com/JHA-Lab/cnn_design-space.git
%cd cnn_design-space/visualization
%ls

Cloning into 'cnn_design-space'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 487 (delta 16), reused 19 (delta 8), pack-reused 442
Receiving objects: 100% (487/487), 223.26 MiB | 26.80 MiB/s, done.
Resolving deltas: 100% (266/266), done.
/content/cnn_design-space/visualization
cnnbench_colab.ipynb  cnnbench_results.ipynb  nasbench_var.ipynb


In [2]:
import json
import os
import re
import tensorflow as tf
import copy
import numpy as np
import random
import sys

if '../' not in sys.path:
    sys.path.append('../')
    
from cnnbench.lib import config as _config
from cnnbench.lib import print_util
from absl import logging

# Set DEBUG to True to print intermediate variables
DEBUG = False

# Define model directory
model_dir = '../results/vertices_2/'

In [3]:
config = _config.build_config()
dataset = None

with open(os.path.join(model_dir, 'generated_graphs.json')) as f:
    models = json.load(f)

if DEBUG: print(f'All hashes: models.keys()')     

In [4]:
loaded_dataset = tf.data.TFRecordDataset(os.path.join(model_dir, 'cnnbench.tfrecord'))

features = {
        'graph_adjacencies': tf.io.FixedLenSequenceFeature([], tf.int64, allow_missing=True),
        'graph_vertices': tf.io.FixedLenSequenceFeature([], tf.int64, allow_missing=True),
        'graph_operations': tf.io.FixedLenFeature([], tf.string),
        'trainable_parameters': tf.io.FixedLenFeature([], tf.int64),
        'training_time': tf.io.FixedLenFeature([], tf.float32),
        'train_accuracy': tf.io.FixedLenFeature([], tf.float32),
        'validation_accuracy': tf.io.FixedLenFeature([], tf.float32),
        'test_accuracy': tf.io.FixedLenFeature([], tf.float32)}

def _parse_image_function(example_proto):
  return tf.io.parse_single_example(example_proto, features)

parsed_dataset = loaded_dataset.map(_parse_image_function)
parsed_dataset
for elem in parsed_dataset:
    graph_adjacency = elem['graph_adjacencies'].numpy()
    dims = elem['graph_vertices'].numpy()
    trainable_parameters = elem['trainable_parameters'].numpy()
    training_time = elem['training_time'].numpy()
    train_accuracy = elem['train_accuracy'].numpy()
    validation_accuracy = elem['validation_accuracy'].numpy()
    test_accuracy = elem['test_accuracy'].numpy()
    graph_operations = elem['graph_operations'].numpy().decode("utf-8")
    print(f'{print_util.bcolors.HEADER}Graph:{print_util.bcolors.ENDC}')
    for module in range(len(dims)):
        print(f'{print_util.bcolors.OKBLUE}Module {module+1}{print_util.bcolors.ENDC}')
        module_adjacency = graph_adjacency[np.square(dims[module])*module:np.square(dims[module])*(module+1)]
        adjacency = np.array([int(e) for e in list(module_adjacency)], dtype=np.int8)
        adjacency = np.reshape(adjacency, (dims[module], dims[module]))
        operations = graph_operations.split(',')[dims[module]*module:dims[module]*(module+1)]
        
    
        print(f'Adjacency matrix: \n{adjacency} \nOperations: {operations}')
    print(f'Trainable parameters: {trainable_parameters} \nTrain Accuracy: {train_accuracy} \nValidation Accuracy: {validation_accuracy} \nTest Accuracy: {test_accuracy}\n')

Graph:
Module 1
Adjacency matrix: 
[[0 1]
 [0 0]] 
Operations: ['input', 'output']
Module 2
Adjacency matrix: 
[[0 1]
 [0 0]] 
Operations: ['input', 'output']
Module 3
Adjacency matrix: 
[[0 1]
 [0 0]] 
Operations: ['input', 'output']
Module 4
Adjacency matrix: 
[[0 1]
 [0 0]] 
Operations: ['input', 'output']
Module 5
Adjacency matrix: 
[[0 1]
 [0 0]] 
Operations: ['input', 'output']
Module 6
Adjacency matrix: 
[[0 1]
 [0 0]] 
Operations: ['input', 'output']
Module 7
Adjacency matrix: 
[[0 1]
 [0 0]] 
Operations: ['input', 'output']
Module 8
Adjacency matrix: 
[[0 1]
 [0 0]] 
Operations: ['input', 'output']
Module 9
Adjacency matrix: 
[[0 1]
 [0 0]] 
Operations: ['input', 'output']
Trainable parameters: 882570 
Train Accuracy: 0.09892857074737549 
Validation Accuracy: 0.10834703594446182 
Test Accuracy: 0.10006009787321091

Graph:
Module 1
Adjacency matrix: 
[[0 1]
 [0 0]] 
Operations: ['input', 'output']
Module 2
Adjacency matrix: 
[[0 1]
 [0 0]] 
Operations: ['input', 'output']
Modul